### Calculate labour income losses

In [1]:
import numpy as np
import pandas as pd
from pandas import isnull
import os, time
import warnings
import sys
import pickle
import copy

In [2]:
print('This is the model for Bay Area')
myCountry = 'BA'
suffix = '_baseline'# options ('_baseline',no_code','retrofit','no_UI','UI_2yrs','no_ins','ins50_15')
nsims = 500
#Levels of index at which one event happens

event_level = ['county', 'hazard', 'rp']
avg_prod_k  = 0.3 # average productivity of capital, value from the global resilience model


# Construct cat_info for the Bay Area separately
if myCountry =='BA':
    raw_data = pd.read_csv('./Input data/Simply analytics data/Output/INPUT_for_resilience_model_v2.csv')
    raw_data_real_estate = pd.read_csv('./Input data/Simply analytics data/Output/INPUT_real_estate_for_resilience_model.csv')
    fault_identifier = 'Hayward_sc28_6'
    
    
    cat_info = pd.DataFrame(raw_data.iloc[:,0:63])
    cat_info = pd.merge(cat_info,raw_data_real_estate,on='tract',how='inner')
    # Drop values where pcwgt ==0 or hhwgt ==0
    cat_info = cat_info.loc[(cat_info.pcwgt!=0) & (cat_info.n_hh!=0) & (cat_info.hh_str_asset_value!=0)]
                                         
#                                         ['county_name','tract','pcwgt','pcinc','pov_frac','pop_dep','hh_share','pcsoc','hh_str_asset_value',
#                 'peinc_AGR','peinc_MIN','peinc_UTI','peinc_CON','peinc_MAN','peinc_WHO','peinc_RET','peinc_TRA',
#                                    'peinc_INF','peinc_FIN','peinc_PRO','peinc_EDU','peinc_ART','peinc_OTH','peinc_GOV',
#                                    'Emp_AGR','Emp_MIN','Emp_UTI','Emp_CON','Emp_MAN','Emp_WHO','Emp_RET','Emp_TRA','Emp_INF','Emp_FIN',
#                                    'Emp_PRO','Emp_EDU','Emp_ART','Emp_OTH','Emp_GOV','Unemp','n_hh','LIL_hh',
#                                    'VLIL_hh','ELIL_hh']])
    cat_info = cat_info.rename(index = str, columns = {'n_hh':'hhwgt','hh_size':'hhsize',
                                                       'hh_str_asset_value':'str_asset_value_tot',
                                                       'county_name':'county',
                                                       'hh_pub_ass_inc':'hh_pub_assist_inc'})
    cat_info['has_ew'] = 0
    ind_names = ['AGR','MIN','UTI','CON','MAN','WHO','RET','TRA','INF','FIN','PRO','EDU','ART','OTH','GOV']

    cat_info = cat_info.set_index(['county','tract'])
    
    
    ## Process the labor income
    cat_info['inc_L_tot'] = 0
    cat_info['pcinc_L'] = 0
    for ind in ind_names:
        # get total industry income
        cat_info['inc_'+ind+'_tot'] = cat_info.loc[:,'peinc_'+ind]*cat_info.loc[:,'Emp_'+ind]
         # add up to the total labour income (all industries)
        cat_info.inc_L_tot += cat_info['inc_'+ind+'_tot']
        # get per capita (pc) industry income
        cat_info['pcinc_'+ind] = cat_info['inc_'+ind+'_tot']/cat_info.pcwgt
        # add up to the total pc labour income (all industries)
        cat_info.pcinc_L += cat_info['pcinc_'+ind]
        
        # round the labour incomes
        cat_info.pcinc_L = cat_info.pcinc_L.round(0)
    
        # drop unnecessary columns
        cat_info = cat_info.drop(columns = 'Emp_'+ind) # drop unnecessary columns
        cat_info = cat_info.drop(columns = 'peinc_'+ind) 
        
    #cat_info = cat_info.drop(columns =['LIL_pc','VLIL_pc','ELIL_pc'])
    ## Variables in previous analyses that are not used anymore
    #cat_info['pcinc_ae'] = None
    #cat_info['hhsize_ae'] = None
    #cat_info['pcwgt_ae'] = None
    #cat_info['quantile'] = None
    #cat_info['c_5'] = None
    #cat_info['gamma_SP'] = None 
    #cat_info['ispoor'] = 0        
    #cat_info['dk'] = None #To add, this is the loss in capital 

    #cat_info['c'] = cat_info.pcinc
    #cat_info['social'] = cat_info.pcsoc/cat_info.c
    
    # Set a floor for pcinc
    cat_info['pcinc_non_adj'] = cat_info['pcinc'].copy()

    # THIS IS FROM PERVIOUS VERSION: Make sure their income is corresponding to the structural value of the houses
    #   cat_info['pcinc'] = pd.concat([cat_info.pcinc,cat_info.str_asset_value_tot*cat_info.hh_share/cat_info.pcwgt*get_avg_prod(myCountry)],axis = 1).max(axis = 1)  

    # Make sure pc income is at least at big as the labour income
    cat_info['pcinc'] =  pd.concat([cat_info.pcinc,cat_info.pcinc_L],axis = 1).max(axis = 1)
    
    # Add all sources of income and calculate the total income
    cat_info['pcinc_oth'] = (cat_info.pcinc-cat_info.pcinc_L)
    #cat_info['pcinc_h'] = cat_info.k_pc_h * get_avg_prod(myCountry)
        # Th pcinc_tot total income includes non-monetary income from housing services
    #cat_info['pcinc_tot'] =cat_info.pcinc_L + cat_info.pcinc_oth + cat_info.pcinc_h - cat_info.p_rent_pc
    
'''   
    # Get productive capital values
    cat_info['k_pc_L'] = cat_info.pcinc_L/get_avg_prod(myCountry)
    cat_info['k_pc_oth'] =  cat_info.pcinc_oth/get_avg_prod(myCountry)
    cat_info['k_pc'] =   cat_info.k_pc_L + cat_info.k_pc_h + cat_info.k_pc_oth
# =============================================================================
## =============================================================================
   
    
    # Get total values, aggregated on a census level
    cat_info['k_tot'] =   cat_info.k_pc*cat_info.pcwgt
    cat_info['savings_tot'] = cat_info.savings_per_hh*cat_info.hhwgt
    cat_info['pension_ss_tot'] = cat_info.pension_ss_pc*cat_info.pcwgt
'''

# Drop unnecessary columns
columns_to_drop =  ['n_hh_inc_less15','n_hh_inc_15_25', 'n_hh_inc_25_35','n_hh_inc_35_50', 'n_hh_inc_50_75',
 'n_hh_inc_75_100', 'n_hh_inc_100_125', 'n_hh_inc_125_150', 'n_hh_inc_150_200', 'n_hh_inc_more200']
    
cat_info = cat_info.drop(columns = columns_to_drop)

#Write the data
print 'DONE WRITING CAT INFO'



This is the model for Bay Area
DONE WRITING CAT INFO


In [3]:
dt = 1./52
n_steps = 520
print dt
print n_steps

0.0192307692308
520


In [4]:
df_inc_loss = pd.DataFrame(index = ind_names)
for ind in ind_names: 
    print ind
    df_inc_loss.loc[ind,'total_income'] = (cat_info['pcinc_'+ind]*cat_info['pcwgt']).sum()/1e9
    df_labour = pd.read_csv('./Output/Employment/Employment_ind_'+ fault_identifier +'_DEC2018_RIMS'+suffix+'/Employment_'+ind+'_'+fault_identifier +'.csv')
    for sim in np.arange(1,nsims+1):
        df_inc_loss.loc[ind,sim]  = sum(df_labour.iloc[sim-1,1:int(n_steps)+1]*dt)*df_inc_loss.loc[ind,'total_income']



AGR
MIN
UTI
CON
MAN
WHO
RET
TRA
INF
FIN
PRO
EDU
ART
OTH
GOV


In [5]:
df_inc_loss.loc['TOT','total_income'] = df_inc_loss.total_income.iloc[0:15].sum()
for sim in np.arange(1,nsims+1):
    df_inc_loss.loc['TOT',sim]  = df_inc_loss.iloc[0:15,sim].sum()

df_inc_loss.to_csv('./Output/Employment/income_losses_'+fault_identifier+suffix+"_"+str(nsims)+"sims.csv")
print df_inc_loss

     total_income         1          2          3          4          5  \
AGR      1.058247  0.003305   0.008091   0.010780   0.082098   0.010799   
MIN      0.260394  0.002332   0.006207   0.007832   0.027939   0.007717   
UTI      2.857232  0.035674   0.110293   0.131965   0.387782   0.128067   
CON     15.314034 -4.529268 -11.151839 -11.393541 -14.149910 -11.592299   
MAN     53.773473 -0.678445  -2.809087  -2.656025  -1.261266  -2.494192   
WHO      7.783529  0.335964   0.823074   1.013293   2.192277   1.020543   
RET     19.041016  0.543584   1.599192   1.866128   4.248821   1.921684   
TRA      7.630370  0.324950   0.916846   1.126268   2.209589   1.082107   
INF     23.777917  0.397039   1.162239   1.409475   3.097101   1.342205   
FIN     29.626113  0.828919   2.759487   3.152648   6.515607   3.044015   
PRO     72.022202  2.495998   7.638331   9.339655  17.741141   8.756643   
EDU     46.641985  3.062326  10.873732  10.965916  19.656298  11.556058   
ART      9.907551  0.3878

In [ ]:
#%%
## Write the hazard_ratios.csv file    
if myCountry =='BA':
    rp = np.arange(1,nsims+1,1)
    
    #%% Create labour income recovery pickle
    x_max = 10 # 10 years
    x_min, n_steps = 0.,52.*x_max # <-- time step = week
    int_dt,step_dt = np.linspace(x_min,x_max,num=n_steps,endpoint=True,retstep=True)
    L_pcinc_dic = {}
    L_recov_dic = {}
    for ind in ind_names:
        df_labour = pd.read_csv('./Output/Employment/Employment_ind_'+ fault_identifier +'_DEC2018_RIMS'+suffix+'/Employment_'+ind+'_'+fault_identifier +'.csv')
        L_pcinc_dic[ind] = cat_info['pcinc_'+ind]
        for sim in rp:
            print 'Ind:',ind,' sim:',sim
            L_recov_dic[(ind,sim)] = np.array(df_labour.iloc[sim-1,1:int(n_steps)+1])
